In [189]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import urllib
import urllib2
import simplejson
import json
import string
import h5py
import string

monlist = pd.read_csv('monitorlist.txt', header=None)[0].tolist()

In [201]:
opendate=lambda x: x['opendate']
netamount=lambda x: x['netamount']
r0_net=lambda x: x['r0_net']

jsonloadfun = lambda x: json.loads(x)
ggzjl = r'http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/MoneyFlow.ssl_qsfx_lscjfb?page=1&num=400&sort=opendate&asc=0&daima='

replacestr = lambda strbad: strbad.replace('{', '{\"').replace(':\"', '\":\"').replace(',', ',\"').replace('\"{', '{')
getdata = lambda stockid: urllib2.urlopen(urllib2.Request(ggzjl+str(stockid))).read()

frame0 = pd.DataFrame()
frame1 = pd.DataFrame()
frame2 = pd.DataFrame()
def gentable(id):
    #global framenet
    global frame0
    global frame1
    global frame2
    tmpresult = map(getdata, [id])
    mapedresult = map(replacestr,tmpresult)
    jsonresult = map(jsonloadfun, mapedresult)
    frame = pd.DataFrame(jsonresult)
    s1 = frame.ix[0]
    s1opendate = s1.apply(opendate)
    frame.applymap(opendate)
    frame.index=[id]
    frame.columns=[s1opendate]
    frame.columns.names = ['opendate']
    
    framenetamount = frame.applymap(netamount).applymap(string.atof)
    framenetamount.index=['netamount']
    framenetamount.columns=[s1opendate]
    framenetamount.columns.names = ['opendate']
    
    
    framer0_net = frame.applymap(r0_net).applymap(string.atof).applymap(lambda x: int(x / 10 ** 4))
    #framer0_net.index=['r0_net']
    framer0_net.index=[id]
    framer0_net.columns=[s1opendate]
    framer0_net.columns.names = ['opendate']
    
    frameall = framer0_net.append(framenetamount)
    frame1 = framenetamount
    frame2 = framer0_net
    
    #print frameall
    if frame0.empty == True:
        frame0 = framer0_net
    else:
        frame0 = frame0.append(framer0_net)
    frame0.T.sort_index(ascending=False).to_csv('monitor' + '.csv')

map(gentable, monlist)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [191]:
frame1

opendate,2016-12-02,2016-12-01,2016-11-30,2016-11-29,2016-11-28,2016-11-25,2016-11-24,2016-11-23,2016-11-22,2016-11-21,...,2015-04-30,2015-04-29,2015-04-28,2015-04-27,2015-04-24,2015-04-23,2015-04-22,2015-04-21,2015-04-20,2015-04-17
netamount,-2.786657e+08,-782776991.0,-6.060348e+08,4.315116e+09,5.654660e+09,6.398898e+08,-8.103764e+08,-9.973003e+08,-7.417949e+08,1.601062e+09,...,8.759091e+09,-3.055708e+08,1.697797e+09,3.158913e+09,1.389315e+09,1.070767e+08,6.016281e+08,1.304635e+09,5.813191e+09,2.105308e+09


In [192]:
frame2

opendate,2016-12-02,2016-12-01,2016-11-30,2016-11-29,2016-11-28,2016-11-25,2016-11-24,2016-11-23,2016-11-22,2016-11-21,...,2015-04-30,2015-04-29,2015-04-28,2015-04-27,2015-04-24,2015-04-23,2015-04-22,2015-04-21,2015-04-20,2015-04-17
sh601668,-30874.653664,-73166.753083,-59211.596797,425303.801705,553512.46748,59169.157663,-70409.957785,-89360.80544,-74201.207741,156406.132118,...,822698.502414,-30669.730624,169709.502328,314896.275802,134362.847324,11661.031596,58632.485788,128587.149824,581504.974472,210448.60137


In [194]:
framecheck = (frame1.T.netamount==frame2.T.sh601668)

In [198]:
framecheck

pandas.core.series.Series

In [176]:
frame1t = frame1.T

In [178]:
frame1t.apply(framecheck)

TypeError: ("'Series' object is not callable", u'occurred at index netamount')

In [147]:
pd.merge(frame0.T, frame1.T, how='outer')

opendate,2016-12-02,2016-12-01
0,570.449968,413.915600
1,-124.450400,-362.566368


In [149]:
frame0.T.append(frame1.T)

opendate,2016-12-02,2016-12-01
sz002676,570.449968,413.915600
sh600452,-124.450400,-362.566368
